In [3]:
import pandas as pd
movie = pd.read_csv("../db/movie_list.csv", index_col=0)
movie

,영화ID,영화명,개봉일,국적,장르,감독,배우
0,0,범죄도시2,2022-05-18,한국,"범죄,액션",이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형..."
1,1,아바타: 물의 길,2022-12-14,미국,"액션,어드벤처,SF,스릴러",제임스 카메론,"조 샐다나,샘 워싱턴,시고니 위버,스티븐 랭,케이트 윈슬렛"
2,2,탑건: 매버릭,2022-06-22,미국,액션,조셉 코신스키,"톰 크루즈,마일즈 텔러,제니퍼 코넬리,존 햄,에드 해리스,글렌 포웰,제이 엘리스,그..."
3,3,스파이더맨: 노 웨이 홈,2021-12-15,미국,"액션,어드벤처,SF",존 왓츠,"톰 홀랜드,젠데이아 콜먼,베네딕트 컴버배치,제이콥 배덜런,존 파브로,마리사 토메이"
4,4,한산: 용의 출현,2022-07-27,한국,"액션,드라마",김한민,"박해일,이지영,이정태,변요한,안성기,손현주,김성규,김성균,김향기,옥택연,공명,박지환..."
...,...,...,...,...,...,...,...
15839,15839,허수아비들의 땅,2009-04-30,한국,드라마,노경태,"김선영,박원주,김화정,김두홍,신예슬,황상연,박현진,서민정,최경락,최광수,윤심경,임독..."
15840,15840,혹성탈출: 진화의 시작,2011-08-17,미국,"액션,SF,드라마",루퍼트 와이어트,"브라이언 콕스,제임스 프랭코,프리다 핀토"
15841,15841,환상의 빛,2016-07-07,일본,드라마,고레에다 히로카즈,"마키코 에수미,나이토 타카시,아사노 타다노부"
15842,15842,후세: 말하지 못한 내 사랑,2013-03-28,일본,"애니메이션,드라마",미야지 마사유키,"코토부키 미나코,미야노 마모루,코니시 카츠유키"


In [4]:
movie_list = movie['영화명']
movie_list

0                  범죄도시2
1              아바타: 물의 길
2                탑건: 매버릭
3          스파이더맨: 노 웨이 홈
4              한산: 용의 출현
              ...       
15839           허수아비들의 땅
15840       혹성탈출: 진화의 시작
15841              환상의 빛
15842    후세: 말하지 못한 내 사랑
15843     흐르는 강물을 어찌 막으랴
Name: 영화명, Length: 15844, dtype: object

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from tqdm import tqdm
from urllib.parse import quote

In [2]:
movie = '범죄도시2'

In [3]:
url = "https://www.naver.com/"
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Edge('edgedriver', options=options)

driver.get(url)
time.sleep(2)

C:\Users\gram15\AppData\Local\Temp\ipykernel_2844\1222300948.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('edgedriver', options=options)


In [4]:
driver.find_element(By.CSS_SELECTOR, '#query').send_keys([f'{movie} 평점', Keys.ENTER])
time.sleep(1)

In [9]:
# 리뷰/평점 리스트에서 관람객으로 설정되어있을 경우 전체 리뷰로 변경
soup = BeautifulSoup(driver.page_source, 'html.parser')

if  soup.select_one('._sorting_select_actual._trigger > a > span.menu._trigger_text').get_text() == '관람객':
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual._trigger').click()
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual > div > div > div > div > ul > li:nth-child(1) > a').click()
else:
    pass

# 스포일러 허용 버튼
driver.find_element(By.CSS_SELECTOR, '._spoiler_switch > button.btn_area_auto').click()

In [ ]:
# 리뷰 전체 내용을 가져오기위해 스크롤을 끝까지 내리는 코드
# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight") 
element = driver.find_element(By.CSS_SELECTOR, "#main_pack > div.sc_new.cs_common_module.case_empasis.color_4._au_movie_content_wrap > div.cm_content_wrap > div.cm_content_area._cm_content_area_rating > div") # 스크롤을 내릴 요소 찾기
# 끝까지 스크롤 다운
while True:
    new_height = element.get_attribute("scrollHeight") # 스크롤 높이 가져오기
    if new_height == last_height: # 스크롤 높이가 같으면 중지
        break
    driver.execute_script("scrollTo(0, .scrollHeight);") # 스크롤 다운
    time.sleep(1) # 1초 대기
    last_height = new_height # 마지막 스크롤 높이 업데이트

In [17]:
# 내부 스크롤 가져오기
inner_height = driver.execute_script("return document.querySelector('#main_pack > div.sc_new.cs_common_module.case_empasis.color_4._au_movie_content_wrap > div.cm_content_wrap > div.cm_content_area._cm_content_area_rating > div > div:nth-child(2) > div.lego_review_list._scroller').scrollHeight")

# 내부 스크롤 다운
while True:
    driver.execute_script("arguments[0].scrollBy(0, inner_height)", driver.find_element(By.CSS_SELECTOR, "._cm_content_area_rating > div > div:nth-child(2) > div.lego_review_list._scroller"))
    time.sleep(0.1)
    new_height = driver.execute_script("return document.querySelector('.lego_review_list._scroller').scrollHeight")
    if new_height == inner_height:
        break
    last_height = new_height

driver.quit()

JavascriptException: Message: javascript error: inner_height is not defined
  (Session info: MicrosoftEdge=112.0.1722.68)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF68C07E082+60274]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF68C00ECE2+818866]
	(No symbol) [0x00007FF68BC8DFAE]
	(No symbol) [0x00007FF68BC91C5D]
	(No symbol) [0x00007FF68BC918D9]
	(No symbol) [0x00007FF68BC92946]
	(No symbol) [0x00007FF68BD09A15]
	(No symbol) [0x00007FF68BCEF56A]
	(No symbol) [0x00007FF68BCC2751]
	(No symbol) [0x00007FF68BD0891C]
	(No symbol) [0x00007FF68BCEF343]
	(No symbol) [0x00007FF68BCC1796]
	(No symbol) [0x00007FF68BCC0975]
	(No symbol) [0x00007FF68BCC1F04]
	Microsoft::Applications::Events::EventProperties::SetLevel [0x00007FF68BF17177+1678103]
	Microsoft::Applications::Events::EventProperties::SetLevel [0x00007FF68BDBEECD+268397]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF68BF4FE87+36951]
	Microsoft::Applications::Events::EventPr

In [140]:
# 리뷰만 추출
review = soup.select_one('ul > li:nth-child(2) > div.area_review_content > div > ._text').get_text()
review

'마동석외에도 연기 좋아요 '

In [129]:
# 평점 요소
rating = soup.select_one('li:nth-child(3) > div.area_title_box > div > div.area_text_box').get_text()
rating

'별점(10점 만점 중)8'

In [128]:
# 맨 뒤에 평점만 추출
rating[-1:]

'8'